In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [18]:
df1 = pd.read_csv("11th_hour_political_tweets.csv", sep='|')

,Unnamed: 0,last_updated,tweet_id,created_at,user_location,full_text,quote_count,reply_count,retweet_count,favorite_count,...,Pos_VERB_count,Neg_VERB_count,Pos_ADV_count,Neg_ADV_count,neg,neu,pos,compound,weighted_compound,weighted_compound_new
0,0,2019-03-06T17:30:21.000+05:30,1.103136e+18,2019-03-06T03:33:25.000+05:30,Guwahati,Guwahati has gained a lot of prominence in rec...,0,4.0,113.0,113.0,...,0.0,0.0,0.0,0.0,0.000,0.880,0.120,0.6486,73.2918,0.000911
1,1,2019-03-17T18:11:58.000+05:30,1.107333e+18,2019-03-17T17:27:57.000+05:30,NaN,RSS in school days itself? Indoctrinated since...,0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.332,0.579,0.088,-0.7906,-3.9530,-0.000303
2,2,2019-03-23T18:07:38.000+05:30,1.109417e+18,2019-03-23T11:28:31.000+05:30,New Delhi,Words don't exist in Vaccum!\nModi's post Pulw...,0,4.0,31.0,31.0,...,0.0,0.0,0.0,0.0,0.130,0.870,0.000,-0.6900,-21.3900,-0.000577
3,3,2019-03-11T17:47:32.000+05:30,1.100300e+18,2019-02-26T07:40:28.000+05:30,NaN,"India's #Icecream Industry stands at 15,000 cr...",0,0.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.000,0.955,0.045,0.1779,0.5337,-0.000232
4,4,2019-02-27T18:18:42.000+05:30,1.100733e+18,2019-02-27T12:21:18.000+05:30,Delhi | Odisha,*40 CRPF Jawans killed in Pulwama Attack\n*IAF...,0,0.0,9.0,9.0,...,0.0,0.0,0.0,0.0,0.362,0.638,0.000,-0.9638,-8.6742,-0.000377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46038,46038,2019-04-07T17:41:43.000+05:30,1.114911e+18,2019-04-07T15:21:43.000+05:30,New Delhi,My article for https://t.co/b8FIn1GYyz today -...,0,0.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.000,1.000,0.000,0.0000,0.0000,-0.000241
46039,46039,2019-04-09T18:10:03.000+05:30,1.115590e+18,2019-04-09T12:19:44.000+05:30,Delhi,Not only in #Noida #homebuyers like #AntrikshS...,0,0.0,2.0,2.0,...,1.0,1.0,0.0,0.0,0.065,0.792,0.144,0.4031,0.8062,-0.000228
46040,46040,2019-03-06T18:09:39.000+05:30,1.103308e+18,2019-03-06T14:55:22.000+05:30,ಹಾಸನ Banglore Karnataka,Modi did nothing in last 5 yrs other than the ...,0,1.0,26.0,26.0,...,0.0,1.0,0.0,0.0,0.105,0.864,0.031,-0.6059,-15.7534,-0.000488
46041,46041,2019-02-08T18:25:06.000+05:30,1.093938e+18,2019-02-08T18:23:22.000+05:30,Others,Congress turns up heat on White House for answ...,0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.155,0.845,0.000,-0.6597,-0.6597,-0.000251


In [63]:
df = pd.read_csv("Twitter Popularity Statistics.csv")

df = df.iloc[1:]
df
df = df.rename(columns={"Unnamed: 1": "Username", "Unnamed: 2": "Retweets",
                    "Unnamed: 3": "Replies", "Unnamed: 4": "Favorites",
                    "Unnamed: 5": "Count", "Unnamed: 6": "Pos", "Unnamed: 7":"Neu",
                    "Unnamed: 8": "Neg"
                   })
df = df.drop(columns=['popularity_df_with_avg_sentiment'])
df.columns
df["Retweets"] = pd.to_numeric(df["Retweets"])
df["Replies"] = pd.to_numeric(df["Replies"])
df

,Username,Retweets,Replies,Favorites,Count,Pos,Neu,Neg
1,narendramodi,1105424.0,53447.0,1105424.0,3407,0.367425091160089,0.215559478673678,0.417015429696266
2,ncbn,11328.0,621.0,11328.0,64,0.241565488628112,0.17330711777322,0.58512740356673
3,mepratap,110.0,12.0,110.0,5,0.370435848087072,0.150103881955147,0.479460247606039
4,Mayor17Ian,0.0,1.0,0.0,1,0.0433916300535202,0.0849843621253967,0.871623992919922
5,marklevinshow,18.0,1.0,18.0,9,0.122837418069442,0.0891200200551086,0.788042564358976
...,...,...,...,...,...,...,...,...
10259,1a1c5cf504b6470,1.0,0.0,1.0,1,0.186195179820061,0.627912402153015,0.18589249253273
10260,farhans17,1.0,0.0,1.0,1,0.186195179820061,0.627912402153015,0.18589249253273
10261,FarrukhRz,1.0,0.0,1.0,1,0.186195179820061,0.627912402153015,0.18589249253273
10262,SidAAP,1.0,0.0,1.0,1,0.96367758512497,0.02154459990561,0.0147778168320655


In [71]:
df.sort_values(by='Retweets', ascending=False)

df["Reachability"]  =  df["Retweets"] +  0.8*df["Replies"]


df["Reachability"]  = df["Reachability"].astype(int)

df["Neg"]  = pd.to_numeric(df["Neg"])
df = df.sort_values(by='Reachability', ascending=False)

for i in list(df[:10]["Reachability"]):
    print(round( i, 2))

df

1148181
613334
347869
186914
186081
101317
91617
67638
63531
52841


,Username,Retweets,Replies,Favorites,Count,Pos,Neu,Neg,Reachability
1,narendramodi,1105424.0,53447.0,1105424.0,3407,0.367425091160089,0.215559478673678,0.417015,1148181
8,RahulGandhi,577011.0,45404.0,577011.0,1726,0.234798223500573,0.25712569688122,0.508076,613334
10,INCIndia,322993.0,31095.0,322993.0,1583,0.241652614599422,0.234847498855266,0.523500,347869
29,AOC,179254.0,9576.0,179254.0,213,0.177443346673264,0.184234102580154,0.638323,186914
13,BJP4India,175769.0,12890.0,175769.0,901,0.299309280066318,0.24835146753041,0.452339,186081
...,...,...,...,...,...,...,...,...,...
6869,indica,0.0,0.0,0.0,1,0.0314500965178012,0.0506743490695953,0.917876,0
6258,BholaNath_BSF,0.0,0.0,0.0,1,0.0310465153306722,0.0497573651373386,0.919196,0
6621,dnyadav,0.0,1.0,0.0,2,0.265584915876389,0.450985863804817,0.283429,0
6927,IcedCinnamon,0.0,0.0,0.0,1,0.594674170017242,0.252273917198181,0.153052,0
